# Pipeline d'entraînement des modèles NER avec spaCy (Google colab / GPU)

Referencias:
*   NER4ARCHIVES : https://github.com/NER4Archives-project/spacy_ner_training_pipeline
*   Spacy transformers : https://www.youtube.com/watch?v=Y_N_AO39rRg
*   Spacy tranformers : https://towardsdatascience.com/how-to-fine-tune-bert-transformer-with-spacy-3-6a90bfe57647
*   Spancat Spacy : https://explosion.ai/blog/spancat


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install spacy-huggingface-hub
!pip install spacy-transformers

In [ ]:
!python -m spacy download fr_dep_news_trf
!python -m spacy download fr_core_news_sm

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.8/397.8 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 43.3 MB/s eta 0:00:00
✔ Download

Vérifier Spacy en version 3.x et l'utilisation du GPU

In [ ]:
import spacy
import torch
torch.cuda.empty_cache()

print('GPU State : ', spacy.prefer_gpu()) # doit renvoyer True
try:
  print('GPU required : ', spacy.require_gpu(0)) # doit renvoyer True
except ValueError:
  print("No GPU devices detected")
print('Cuda available : ', torch.cuda.is_available()) # doit renvoyer True
print('SPACY VERSION : ', spacy.__version__)

GPU State :  False
No GPU devices detected
Cuda available :  False
SPACY VERSION :  3.7.4


In [ ]:
!python -m spacy info

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        fr_dep_news_trf (3.7.2), fr_core_news_sm (3.7.0), en_core_web_sm (3.7.1)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1_2etier_modif.conll" "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/bin_containers/" -t spacy -c iob -b fr_core_news_sm


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
⚠ No sentence boundaries found. Use `-s` to automatically segment
sentences.
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Generated output file (53 documents): /content/drive/MyDrive/Colab
Notebooks/TopEditor/data/bin_containers/annot_data_topeditor_1_2etier_modif.spacy


In [ ]:
**!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catTrainDS_project-3-at-2023-12-14-23-16-3e7d0880_modif_ALL.spacy" ./train.spacy

**!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catEvalDS_project-3-at-2023-12-15-12-38-8e39bcf8_modif.spacy" ./dev.spacy

In [ ]:
!python -m spacy init fill-config "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/config/base_config_fr_trf.cfg" ./config.cfg


In [ ]:
!python -m spacy debug config config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
!python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
!python -m spacy train --gpu-id 0 config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catPLO_gpu_trf_es_ALL/"

**Modelo 3 categorias Loc (asentamientos, naturel, territorios)**

In [ ]:
!cat "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catLoc_trainDS_project-4-at-2023-12-26-09-57-88cd17ba_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_A_project-10-at-2024-01-30-23-14-e7d97da2_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_A_project-10-at-2024-01-30-23-14-e7d97da2_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_B_project-11-at-2024-01-30-23-18-00c140c4_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_C_project-12-at-2024-01-30-23-20-d7dc64a1_modif.conll" > "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_ALL_ABC_project-12-at-2024-01-30-23-20-d7dc64a1_modif.conll"

In [ ]:
#!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catLoc_trainDS_project-4-at-2023-12-26-09-57-88cd17ba_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_ALL_ABC_project-12-at-2024-01-30-23-20-d7dc64a1_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

#!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catLoc_evalDS_project-4-at-2023-12-26-09-37-1e54cf98_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

In [ ]:
#!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catLoc_trainDS_project-4-at-2023-12-26-09-57-88cd17ba_modif.spacy" ./train.spacy

!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/LocCat_testDS_ALL_ABC_project-12-at-2024-01-30-23-20-d7dc64a1_modif.spacy" ./train.spacy

!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catLoc_evalDS_project-4-at-2023-12-26-09-37-1e54cf98_modif.spacy" ./dev.spacy

In [ ]:
#!python -m spacy init fill-config "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/config/base_config_es_trf.cfg" ./config.cfg
!python -m spacy init fill-config "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/config/config_gpu_trf_es_config.cfg" ./config.cfg

In [ ]:
!python -m spacy debug config config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
!python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
!python -m spacy train --gpu-id 0 config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catLoc_gpu_trf_es_drp05/"